# Práctica 2: Transformaciones geométricas

### 1a. Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente.

#### Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.

#### Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.

#### Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de escala.

In [ ]:
import cv2 as cv
import numpy as np

# ======================================
# CARGA DE IMAGEN
# ======================================
img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

h, w = img.shape[:2]
size = (w, h)

def nothing(x):
    pass

# ======================================
# CREAR UNA SOLA VENTANA CON TODOS LOS CONTROLES
# ======================================
cv.namedWindow("Transformaciones")

# --- Traslación ---
cv.createTrackbar('tx', 'Transformaciones', w, 2 * w, nothing)
cv.createTrackbar('ty', 'Transformaciones', h, 2 * h, nothing)

# --- Rotación ---
cv.createTrackbar('Angulo', 'Transformaciones', 0, 360, nothing)
cv.createTrackbar('Centro X', 'Transformaciones', w // 2, w, nothing)
cv.createTrackbar('Centro Y', 'Transformaciones', h // 2, h, nothing)

# --- Escalado ---
cv.createTrackbar('Uniforme', 'Transformaciones', 0, 1, nothing)
cv.createTrackbar('Escalado X', 'Transformaciones', 100, 300, nothing)
cv.setTrackbarMin('Escalado X', 'Transformaciones', 10)
cv.createTrackbar('Escalado Y', 'Transformaciones', 100, 300, nothing)
cv.setTrackbarMin('Escalado Y', 'Transformaciones', 10)

# ======================================
# BUCLE PRINCIPAL
# ======================================
while True:
    # ----- Lectura de trackbars -----
    tx = cv.getTrackbarPos('tx', 'Transformaciones') - w
    ty = cv.getTrackbarPos('ty', 'Transformaciones') - h
    ang = cv.getTrackbarPos('Angulo', 'Transformaciones')
    cx = cv.getTrackbarPos('Centro X', 'Transformaciones')
    cy = cv.getTrackbarPos('Centro Y', 'Transformaciones')
    sx = cv.getTrackbarPos('Escalado X', 'Transformaciones') / 100
    sy = cv.getTrackbarPos('Escalado Y', 'Transformaciones') / 100
    uniforme = cv.getTrackbarPos('Uniforme', 'Transformaciones')

    # Escalado uniforme (si está activado)
    if uniforme == 1:
        sx = sy

    # ======================================
    # COMBINACIÓN DE TRANSFORMACIONES
    # ======================================

    # 1️⃣ Matriz de rotación (incluye el centro)
    R = cv.getRotationMatrix2D((cx, cy), ang, 1.0)

    # 2️⃣ Aplicar rotación a la imagen original
    rotated = cv.warpAffine(img, R, size)

    # 3️⃣ Matriz de escalado
    S = np.float32([[sx, 0, 0],
                    [0, sy, 0]])

    scaled = cv.warpAffine(rotated, S, size)

    # 4️⃣ Matriz de traslación
    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])

    # 5️⃣ Aplicar traslación al resultado final
    final = cv.warpAffine(scaled, T, size)


    tipo = " (UNIFORME)" if uniforme == 1 else " (NO UNIFORME)"
    info = f"Ang={ang}° | C=({cx},{cy}) | Tx={tx:+d}, Ty={ty:+d} | Sx={sx:.2f}, Sy={sy:.2f}{tipo}"
    #cv.putText(final, info, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv.imshow("Transformaciones", final)

    # Salir con ESC
    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()
cv.waitKey(1)


### 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [ ]:
# Proyección de una imagen en otra mediante homografía

import cv2 as cv
import numpy as np

# ------------------------------
# Cargar imagen original
# ------------------------------
img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

h, w = img.shape[:2]
img2 = np.ones((h, w, 3), dtype=np.uint8) * 255  # segunda ventana en blanco

# Copias para dibujar los puntos
clone1 = img.copy()
clone2 = img2.copy()

# Listas para guardar puntos
pts_src = []
pts_dst = []

# ------------------------------
# Callback del ratón para ambas ventanas
# ------------------------------
def seleccionar_puntos(event, x, y, flags, param):
    global pts_src, pts_dst, clone1, clone2

    # param = 1 → primera imagen (fuente)
    # param = 2 → segunda imagen (destino)
    if event == cv.EVENT_LBUTTONDOWN:
        if param == 1 and len(pts_src) < 4:
            pts_src.append((x, y))
            cv.circle(clone1, (x, y), 5, (0, 0, 255), -1)
            cv.putText(clone1, f"{len(pts_src)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv.imshow('Original', clone1)

        elif param == 2 and len(pts_dst) < 4:
            pts_dst.append((x, y))
            cv.circle(clone2, (x, y), 5, (255, 0, 0), -1)
            cv.putText(clone2, f"{len(pts_dst)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            cv.imshow('Destino', clone2)

        # Si ya hay 4 puntos en cada imagen, proyectar
        if len(pts_src) == 4 and len(pts_dst) == 4:
            proyectar_imagen()

# ------------------------------
# Función para realizar la proyección
# ------------------------------
def proyectar_imagen():
    global pts_src, pts_dst, img, img2

    # Calcular matriz de transformación de perspectiva
    M = cv.getPerspectiveTransform(np.float32(pts_src), np.float32(pts_dst))

    # Aplicar transformación a la imagen original
    proyectada = cv.warpPerspective(img, M, (img2.shape[1], img2.shape[0]))

    # Mostrar la proyección
    cv.imshow('Proyeccion', proyectada)
    print("Transformación completada.")
    cv.waitKey(0)
    cv.destroyAllWindows()

# ------------------------------
# Ventanas e interacción
# ------------------------------
cv.namedWindow('Original')
cv.namedWindow('Destino')

cv.setMouseCallback('Original', seleccionar_puntos, 1)
cv.setMouseCallback('Destino', seleccionar_puntos, 2)

print("Haz clic en 4 puntos en la imagen ORIGINAL (rojo).")
print("Luego haz clic en 4 puntos en la imagen DESTINO (azul).")

cv.imshow('Original', img)
cv.imshow('Destino', img2)


while True:
    k = cv.waitKey(1) & 0xFF
    if k == 27:  
        break

cv.destroyAllWindows()

# TODO: arreglar cierre robusto


Haz clic en 4 puntos en la imagen ORIGINAL (rojo).
Luego haz clic en 4 puntos en la imagen DESTINO (azul).
Transformación completada.


### 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz

In [ ]:
import cv2
import numpy as np

img = cv.imread('images/eii.png')
size = (img.shape[1], img.shape[0])
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

def nothing(x):
    pass

def apply_distortion(image, k1, k2):
    h, w = image.shape[:2]
    
    distCoeff = np.zeros((4,1), np.float64)
    distCoeff[0,0] = k1  
    distCoeff[1,0] = k2  

    # Suponer matriz unitaria para la cámara
    cam = np.eye(3, dtype=np.float32)
    cam[0,2] = w/2.0  # Definir centro x
    cam[1,2] = h/2.0  # Definir centro y
    cam[0,0] = 10.    # Definir longitud focal x
    cam[1,1] = 10.    # Definir longitud focal y

    # Aplicar distorsión a la imagen
    distorted_img = cv2.undistort(image, cam, distCoeff)

    return distorted_img

h, w = img.shape[:2]

cv2.namedWindow("Distorsion")

cv2.createTrackbar('Barril/Almohadilla', 'Distorsion', 0, 1, nothing)
cv2.createTrackbar('K1', 'Distorsion', 0, 10, nothing)
cv2.createTrackbar('K2', 'Distorsion', 0, 10, nothing)

while True:
    modo = cv2.getTrackbarPos('Barril/Almohadilla', 'Distorsion')
    k1 = cv2.getTrackbarPos('K1', 'Distorsion')
    k2 = cv2.getTrackbarPos('K2', 'Distorsion')
    
    if modo == 0:
        k1 = k1 / 100000
        k2 = k2 / 100000
    else:
        k1 = k1 / -100000
        k2 = k2 / -100000
    
    distorsion_img = apply_distortion(img, k1, k2)
    
    
    cv2.imshow("Original", img)
    cv2.imshow("Distorsion", distorsion_img)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()


### (Optativo – 5 puntos)
- Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en
una imagen destino y realizar la transformación afín.
- Calcular la imagen especular a partir de una imagen.
- Tratar una recta que será el eje de reflexión y “reflejar” la imagen.
- Otras aportaciones

- Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen.

In [5]:
#Hacer Traslaciones
import cv2 as cv
import numpy as np

video = cv.VideoCapture('videos/times-square.mp4')

if not video.isOpened():
    print("Error: no se pudo abrir el video")
    exit()
    
ret, frame = video.read()
if not ret:
    print("Error: video vacío")
    exit()

size = (frame.shape[1], frame.shape[0])
w, h = size

def nothing(x):
    pass

cv.namedWindow("Traslacion")
cv.createTrackbar('tx', 'Traslacion', w, 2*w, nothing)
cv.createTrackbar('ty', 'Traslacion', h, 2*h, nothing)

while True:
    ret, frame = video.read()
    
    if not ret:
            video.set(cv.CAP_PROP_POS_FRAMES, 0)
            continue

    tx = cv.getTrackbarPos('tx', 'Traslacion') - w
    ty = cv.getTrackbarPos('ty', 'Traslacion') - h
    
    T = np.float32([[1,0,tx],
                    [0,1,ty]])

    videoT = cv.warpAffine(frame, T, (w, h))
    vidT = cv.resize(videoT, (800, 400))
    
    info_text = f"Traslacion: X={tx:+4d} px, Y={ty:+4d} px"
    cv.putText(vidT, info_text, (10, 30), 
                cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv.imshow('Traslacion', vidT)


    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()
cv.waitKey(1)

-1

In [9]:
#Hacer Rotaciones
import cv2 as cv
import numpy as np

video = cv.VideoCapture('videos/times-square.mp4')

if not video.isOpened():
    print("Error: no se pudo abrir el video")
    exit()
    
ret, frame = video.read()
if not ret:
    print("Error: video vacío")
    exit()

size = (frame.shape[1], frame.shape[0])
w, h = size

def nothing(x):
    pass

cv.namedWindow("Rotacion")
cv.createTrackbar('Angulo', 'Rotacion', 0, 360, nothing)
cv.createTrackbar('Centro X', 'Rotacion', 0, 200, nothing)
cv.createTrackbar('Centro Y', 'Rotacion', 0, 200, nothing)

while True:
    ret, frame = video.read()
    
    if not ret:
            video.set(cv.CAP_PROP_POS_FRAMES, 0)
            continue
    
    ang = cv.getTrackbarPos('Angulo', 'Rotacion')
    cx = cv.getTrackbarPos('Centro X', 'Rotacion')
    cy = cv.getTrackbarPos('Centro Y', 'Rotacion')
    
    a = np.deg2rad(ang)
    
    T = np.float32(
        [[np.cos(a),-np.sin(a),cx],
         [np.sin(a),np.cos(a),cy]])

    videoR = cv.warpAffine(frame, T, size)
    vidR = cv.resize(videoR, (800, 400))
    
    info_text = f"Angulo: {ang} grados | Centro: ({cx}, {cy})"
    cv.putText(vidR, info_text, (10, 30), 
                cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    cv.imshow('Rotacion', vidR)


    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()
cv.waitKey(1)

-1

In [6]:
#Hacer escalados uniformes y no uniformes
import cv2 as cv
import numpy as np

video = cv.VideoCapture('videos/times-square.mp4')
video 

if not video.isOpened():
    print("Error: no se pudo abrir el video")
    exit()
    
ret, frame = video.read()
if not ret:
    print("Error: video vacío")
    exit()

size = (frame.shape[1], frame.shape[0])
w, h = size


def nothing(x):
    pass

cv.namedWindow("Escalados")
cv.createTrackbar('Uniforme', 'Escalados', 0, 1, nothing)
cv.createTrackbar('Escalado X', 'Escalados', 100, 300, nothing)
cv.setTrackbarMin('Escalado X', 'Escalados', 10)
cv.createTrackbar('Escalado Y', 'Escalados', 100, 300, nothing)
cv.setTrackbarMin('Escalado Y', 'Escalados', 10)

while True:
    ret, frame = video.read()
    
    if not ret:
            video.set(cv.CAP_PROP_POS_FRAMES, 0)
            continue
        
    sx = cv.getTrackbarPos('Escalado X', 'Escalados') / 100
    sy = cv.getTrackbarPos('Escalado Y', 'Escalados') / 100
    uniforme = cv.getTrackbarPos('Uniforme', 'Escalados')
    
    if uniforme == 1:
        sx = sy
    
    T = np.float32(
        [[sx, 0, 0],
         [0, sy, 0]])

    videoE = cv.warpAffine(frame, T, size)
    vidE = cv.resize(videoE, (800, 400))
    
    info_text = f"Sx={sx:.2f}, Sy={sy:.2f}"
    tipo = " (UNIFORME)" if uniforme == 1 else " (NO UNIFORME)"

    cv.putText(vidE, info_text + tipo, (10, 30), 
                cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv.imshow('Escalados', vidE)


    if cv.waitKey(1) & 0xFF == 27:
        break


cv.destroyAllWindows()
cv.waitKey(1)


-1

In [21]:
import cv2
import numpy as np

video = cv.VideoCapture('videos/times-square.mp4')
video 

if not video.isOpened():
    print("Error: no se pudo abrir el video")
    exit()
    
ret, frame = video.read()
if not ret:
    print("Error: video vacío")
    exit()

size = (frame.shape[1], frame.shape[0])
w, h = size

def nothing(x):
    pass

def apply_distortion(image, k1, k2):
    h, w = image.shape[:2]
    
    distCoeff = np.zeros((4,1), np.float64)
    distCoeff[0,0] = k1  
    distCoeff[1,0] = k2  

    # Suponer matriz unitaria para la cámara
    cam = np.eye(3, dtype=np.float32)
    cam[0,2] = w/2.0  # Definir centro x
    cam[1,2] = h/2.0  # Definir centro y
    cam[0,0] = 10.    # Definir longitud focal x
    cam[1,1] = 10.    # Definir longitud focal y

    # Aplicar distorsión a la imagen
    distorted_img = cv2.undistort(image, cam, distCoeff)

    return distorted_img


cv2.namedWindow("Distorsion")

cv2.createTrackbar('Barril/Almohadilla', 'Distorsion', 0, 1, nothing)
cv2.createTrackbar('K1', 'Distorsion', 0, 10, nothing)
cv2.createTrackbar('K2', 'Distorsion', 0, 10, nothing)

while True:
    ret, frame = video.read()
    
    if not ret:
            video.set(cv.CAP_PROP_POS_FRAMES, 0)
            continue
    
    modo = cv2.getTrackbarPos('Barril/Almohadilla', 'Distorsion')
    k1 = cv2.getTrackbarPos('K1', 'Distorsion')
    k2 = cv2.getTrackbarPos('K2', 'Distorsion')
    
    if modo == 0:
        k1 = k1 / 100000
        k2 = k2 / 100000
    else:
        k1 = k1 / -100000
        k2 = k2 / -100000
    
    distorsion_vid = apply_distortion(frame, k1, k2)
    distorsion_resize = cv.resize(distorsion_vid, (800, 400))
    
    cv2.imshow("Distorsion", distorsion_resize)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
cv.waitKey(1)


-1

- Marcar el punto de giro con el ratón.

In [22]:
import cv2 as cv
import numpy as np

img = cv.imread('images/eii.png')
size = (img.shape[1], img.shape[0])
h, w = img.shape[:2]
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

def nothing(x):
    pass

cx = 0
cy = 0

def point_turn(event, x, y, flags, param):
    global cx, cy, img
    
    if event == cv.EVENT_LBUTTONDOWN:
        cx, cy = x, y


cv.namedWindow("Rotacion")
cv.setMouseCallback("Rotacion", point_turn)
cv.createTrackbar('Angulo', 'Rotacion', 0, 360, nothing)

while True:
    ang = cv.getTrackbarPos('Angulo', 'Rotacion')
    
    a = np.deg2rad(ang)
    
    T = np.float32(
        [[np.cos(a),-np.sin(a),cx],
         [np.sin(a),np.cos(a),cy]])

    imageR = cv.warpAffine(img, T, size)
    
    cv.circle(imageR, (cx, cy), 2, (0, 255, 255), 2)
    info_text = f"Angulo: {ang} | Centro: ({cx}, {cy})"
    cv.putText(imageR, info_text, (10, 30), 
                cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    cv.imshow('Rotacion', imageR)


    if cv.waitKey(1) & 0xFF == 27:
        break
    elif cv.waitKey(1) & 0xFF == ord('r') or cv.waitKey(1) & 0xFF == ord('R'):
        cx = 0
        cy = 0

cv.destroyAllWindows()
cv.waitKey(1)

#TODO : creo que el punto no se fija correctamente

-1

- Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real.

In [18]:
#Hacer Traslaciones con el raton
import cv2 as cv
import numpy as np

img = cv.imread('images/eii.png')
size = (img.shape[1], img.shape[0])
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

def nothing(x):
    pass

h, w = img.shape[:2]

draw = False
ix = 0
iy = 0
tx = 0
ty = 0

def move_image(event, x, y, flags, param):
    global draw, ix, iy, tx, ty, img
    
    if event == cv.EVENT_LBUTTONDOWN:
        draw = True
        ix = x - tx
        iy = y - ty
        
    elif event == cv.EVENT_MOUSEMOVE:
        if draw:
            tx = x - ix
            ty = y - iy
        
    elif event == cv.EVENT_LBUTTONUP:
        draw = False


cv.namedWindow("Traslacion con el raton")
cv.setMouseCallback("Traslacion con el raton", move_image)

while True:
    T = np.float32([[1,0,tx],
                    [0,1,ty]])

    imageT = cv.warpAffine(img, T, (w, h))
    
    cv.imshow('Traslacion con el raton', imageT)

    if cv.waitKey(1) & 0xFF == 27:
        break
    elif cv.waitKey(1) & 0xFF == ord('r') or cv.waitKey(1) & 0xFF == ord('R'):
        tx = 0
        ty = 0

cv.destroyAllWindows()
cv.waitKey(1)


-1